CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect
!pip install -U pytorch_tabular[extra]

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Keerthana\\anaconda3\\New folder\\Lib\\site-packages\\~5mpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-65e29aac85b9409a6008e2dc84b1cc09.dll'
Consider using the `--user` option or check the permissions.




  Using cached numpy-1.20.3-cp39-cp39-win_amd64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Keerthana\\anaconda3\\New folder\\Lib\\site-packages\\~8rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

from sklearn.metrics import mean_squared_error, r2_score
import math

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [3]:
# Load the data
df = pd.read_csv('hdb_price_prediction.csv')


train_data = df[df['year'] <= 2019]
test_data_2022 = df[df['year'] == 2022]

target = ['resale_price']
continuous_features = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 
                       'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
categorical_features = ['month', 'town', 'flat_model_type', 'storey_range']

data_config = DataConfig(
    target=['resale_price'], 
    continuous_cols=continuous_features, 
    categorical_cols=categorical_features
)

trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=50,
    batch_size=1024
)

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50", 
    activation="ReLU",
    learning_rate=0.001
)

optimizer_config = OptimizerConfig(optimizer="Adam")

model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

model.fit(train=train_data)

predictions_2022 = model.predict(test_data_2022)
predictions_array_2022 = predictions_2022['resale_price_prediction'].values
test_r2_2022 = r2_score(test_data_2022['resale_price'], predictions_array_2022)
print(f"Test R2 (2022): {test_r2_2022:.2f}")

2023-10-13 18:53:02,825 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-10-13 18:53:02,861 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 18:53:02,865 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 18:53:02,951 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-13 18:53:03,026 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at C:\Users\Keerthana\SC4001 Assignment 1\.lr_find_258c0506-ba72-459f-95fa-bb42fdec384f.ckpt
Restored all states from the checkpoint file at C:\Users\Keerthana\SC4001 Assignment 1\.lr_find_258c0506-ba72-459f-95fa-bb42fdec384f.ckpt
2023-10-13 18:53:06,830 - {pytorch_tabular.tabular_model:575} - INFO - Suggested LR

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-13 18:53:42,558 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 18:53:42,559 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model


Output()

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\utilities\cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


Test R2 (2022): 0.45


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [4]:
# Load the data
df = pd.read_csv('hdb_price_prediction.csv')


train_data = df[df['year'] <= 2019]
test_data_2023 = df[df['year'] == 2023]

target = ['resale_price']
continuous_features = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 
                       'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
categorical_features = ['month', 'town', 'flat_model_type', 'storey_range']

data_config = DataConfig(
    target=['resale_price'], 
    continuous_cols=continuous_features, 
    categorical_cols=categorical_features
)

trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=50,
    batch_size=1024
)

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50", 
    activation="ReLU",
    learning_rate=0.001
)

optimizer_config = OptimizerConfig(optimizer="Adam")

model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)

model.fit(train=train_data)

predictions_2023 = model.predict(test_data_2023)
predictions_array_2023 = predictions_2023['resale_price_prediction'].values
test_r2_2023 = r2_score(test_data_2023['resale_price'], predictions_array_2023)
print(f"Test R2 (2023): {test_r2_2023:.2f}")


2023-10-13 18:53:44,059 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-10-13 18:53:44,110 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 18:53:44,115 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 18:53:44,249 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-13 18:53:44,284 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at C:\Users\Keerthana\SC4001 Assignment 1\.lr_find_0b356c3c-4cab-4dcd-a10a-be898e95dd3e.ckpt
Restored all states from the checkpoint file at C:\Users\Keerthana\SC4001 Assignment 1\.lr_find_0b356c3c-4cab-4dcd-a10a-be898e95dd3e.ckpt
2023-10-13 18:53:48,511 - {pytorch_tabular.tabular_model:575} - INFO - Suggested LR

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-13 18:54:27,165 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 18:54:27,165 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model


Output()

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\utilities\cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


Test R2 (2023): 0.17


> Did model degradation occur for the deep learning model?


\# TODO: \<Enter your answer here\>

Yes, model degradation occurred for the deep learning model when predicting on data from 2022 and 2023. The model's ability to generalize to new data points decreased, which is likely due to the differences in the data distribution between the years 2021, 2022, and 2023

Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [6]:
from alibi_detect.cd import TabularDrift
import numpy as np

np.random.seed(42)  

# Sample 1000 data points from train and test data
train_sample = train_data.sample(1000, random_state=SEED)
test_sample_2023 = test_data_2023.sample(1000, random_state=SEED)

# Define the features to be checked for drift
feature = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 
                     'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']

# Extract the reference and test data as numpy arrays
reference_data = train_sample[feature].values
test_data = test_sample_2023[feature].values

# Create TabularDrift detector
detector = TabularDrift(reference_data, p_val=.05)

# Detect drift on the 2023 test sample
preds = detector.predict(test_data, drift_type='feature', return_p_val=True)

# Get the features that drifted (features with a p-value less than 0.05)
drifted_features = np.array(feature)[preds['data']['p_val'] < 0.05]
print("Drifted Features in 2023 Test Data:", drifted_features)

C:\Users\Keerthana\AppData\Roaming\Python\Python39\site-packages\alibi_detect\cd\tabular.py:113: UserWarning: No `categories_per_feature` dict provided so all features are assumed to be numerical. `KSDrift` will be applied to all features.
  warnings.warn('No `categories_per_feature` dict provided so all features are assumed to be numerical. '


Drifted Features in 2023 Test Data: ['remaining_lease_years' 'floor_area_sqm']


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Enter your answer here\>

The model degradation likely occurred due to label shift in the relationship between features and resale_price. In this case, the housing measures introduced by the government significantly influenced housing prices, causing a shift in the relationship between the features (such as remaining_lease_years and floor_area_sqm) and the resale_price. As a result, the model, which was trained on data up to 2019 and assumed a certain relationship between features and resale_price, performed poorly when predicting on the 2022 and 2023 data than on 2021 data, where this relationship had changed due to external factors. 

> From your analysis via TabularDrift, which features contribute to this shift?


\# TODO: \<Enter your answer here\>

From the analysis via TabularDrift, the features that cotributed to this shift were 'remaining_lease_years', 'floor_area_sqm'.

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\# TODO: \<Enter your answer here\>

One way to address model degradation is by adjusting the model architecture to have more layers and neurons: 128-64-32-16-8. This increases the complexity of the model, allowing it to learn more intricate relationships in the data.
Also, The maximum number of training epochs has been increased to 200. This provides more opportunities for the model to learn from the data.

In [18]:
df = pd.read_csv('hdb_price_prediction.csv')

# Define target variable and features
target = ['resale_price']
continuous_features = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 
                       'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
categorical_features = ['month', 'town', 'flat_model_type', 'storey_range']

# Create data configuration
data_config = DataConfig(
    target=target, 
    continuous_cols=continuous_features, 
    categorical_cols=categorical_features
)

# Create model configuration
model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="128-64-32-16-8",  # You can adjust architecture here
    activation="ReLU",
    learning_rate=0.001,
)

# Create optimizer configuration
optimizer_config = OptimizerConfig(optimizer="Adam")

# Create trainer configuration
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=200,
    batch_size=1024,
)

# Create TabularModel
model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Train the model
model.fit(train=df)

# Evaluate on 2023 data
test_data_2023 = df[df['year'] == 2023]
predictions_2023 = model.predict(test_data_2023)
predictions_array_2023 = predictions_2023['resale_price_prediction'].values
test_r2_2023 = r2_score(test_data_2023['resale_price'], predictions_array_2023)

print(f"Improved Test R2 (2023): {test_r2_2023:.2f}")

2023-10-11 18:28:35,703 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-10-11 18:28:35,753 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-11 18:28:35,761 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-11 18:28:35,983 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-11 18:28:36,021 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\core\module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.09120108393559097
Restoring states from the checkpoint path at C:\Users\Keerthana\SC4001 Assignment 1\.lr_find_a74ff064-2e42-4ab6-aa58-c0c34c602582.ckpt
Restored all states from the checkpoint file at C:\Users\Keerthana\SC4001 Assignment 1\.lr_find_a74ff064-2e42-4ab6-aa58-c0c34c602582.ckpt
2023-10-11 18:28:40,054 - {pytorch_tabular.tabular_model:575} - INFO - Suggested L

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │ 18.7 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.7 K │
│ 2 │ head             │ LinearHead                │      9 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 20.4 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 20.4 K                                                                                               
Total estimated model params size (MB): 0

Output()

2023-10-11 18:31:16,317 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-11 18:31:16,322 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model


Output()

C:\Users\Keerthana\anaconda3\New folder\lib\site-packages\pytorch_lightning\utilities\cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


Improved Test R2 (2023): 0.75


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |